# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123573e+02     1.512100e+00
 * time: 0.0862269401550293
     1     1.015483e+01     8.798207e-01
 * time: 1.3467769622802734
     2    -1.250566e+01     9.709591e-01
 * time: 1.455164909362793
     3    -3.441033e+01     7.581793e-01
 * time: 1.605314016342163
     4    -4.798777e+01     6.198035e-01
 * time: 1.7708539962768555
     5    -5.716032e+01     2.213901e-01
 * time: 1.9078071117401123
     6    -5.989309e+01     1.313613e-01
 * time: 2.0112719535827637
     7    -6.094440e+01     6.036288e-02
 * time: 2.116086006164551
     8    -6.136732e+01     8.387522e-02
 * time: 2.2326090335845947
     9    -6.165863e+01     3.553010e-02
 * time: 2.337161064147949
    10    -6.186969e+01     2.415245e-02
 * time: 2.443087100982666
    11    -6.202741e+01     1.929941e-02
 * time: 2.551358938217163
    12    -6.208951e+01     1.722035e-02
 * time: 2.666785955429077
    13    -6.215216e+01     1.166474e-02
 * time: 2.7756450176239014


In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671074
    AtomicLocal         -18.8557723
    AtomicNonlocal      14.8522676
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485386 
    Xc                  -19.3336825

    total               -62.261666457026
